In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('barueri_dengue_filtered.csv').drop('data inicial semana', axis=1).drop('month', axis=1)

# Check for NaN or infinite values
print(data.isna().sum())
# print(np.isfinite(data).all())

umidmin               13
umidmed               13
umidmax               13
tempmin                0
tempmed               13
tempmax               13
precipitacao total     0
casos                  0
dtype: int64


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv('barueri_dengue_filtered.csv').drop('data inicial semana', axis=1).drop('month', axis=1)

data_filled = data.fillna(data.mean())

X, Y = data_filled.drop('casos', axis=1), data_filled['casos']

X_dev, X_test, Y_dev, Y_test = train_test_split(X, Y, test_size=0.7, random_state=3)
X_train, X_val, Y_train, Y_val = train_test_split(X_dev, Y_dev, test_size=0.5, random_state=3)

x_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(Y_test.values)
x_val = torch.FloatTensor(X_val.values)
y_val = torch.FloatTensor(Y_val.values)
x_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(Y_train.values)



In [10]:
class Casos(nn.Module):
    def __init__(self, input_dim):
        super(Casos, self).__init__()
        self.rede = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.Tanh(),
            nn.Linear(512, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.rede(x)

In [11]:
input_dim = X.shape[1]
model = Casos(input_dim)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
def train_network(model, optimizer, loss_function, x_train, y_train, x_val, y_val, num_epochs, train_losses, val_losses):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Forward pass
        output_train = model(x_train)

        # Calculate training loss
        loss_train = loss_function(output_train, y_train.view(-1, 1))  # Reshape y_train

        # Backward pass
        loss_train.backward()

        # Update weights
        optimizer.step()

        # Validation phase
        output_val = model(x_val)
        loss_val = loss_function(output_val, y_val.view(-1, 1))  # Reshape y_val

        # Store losses
        train_losses[epoch] = loss_train.item()
        val_losses[epoch] = loss_val.item()

        # Print progress every 500 epochs
        if (epoch + 1) % 500 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Val Loss: {loss_val.item():.4f}')

In [ ]:
num_epochs = 15000
train_losses = np.zeros(num_epochs)
val_losses = np.zeros(num_epochs)
train_network(model, optimizer, criterion, x_train, y_train, x_val, y_val, num_epochs, train_losses, val_losses)

Epoch 500/10000, Train Loss: 22963.4492, Val Loss: 19249.8047
Epoch 1000/10000, Train Loss: 22647.9414, Val Loss: 19082.3164
Epoch 1500/10000, Train Loss: 22625.8359, Val Loss: 19104.3457
Epoch 2000/10000, Train Loss: 22625.4121, Val Loss: 19110.4766
Epoch 2500/10000, Train Loss: 19049.7051, Val Loss: 16740.5977
Epoch 3000/10000, Train Loss: 16789.5332, Val Loss: 15427.8467
Epoch 3500/10000, Train Loss: 15301.3506, Val Loss: 14976.3154
Epoch 4000/10000, Train Loss: 14214.4404, Val Loss: 14854.4248
Epoch 4500/10000, Train Loss: 13379.3564, Val Loss: 14633.6279
Epoch 5000/10000, Train Loss: 13211.6211, Val Loss: 16525.1211
Epoch 5500/10000, Train Loss: 11840.7832, Val Loss: 15446.6289
Epoch 6000/10000, Train Loss: 9898.4395, Val Loss: 13195.1182
Epoch 6500/10000, Train Loss: 9099.7041, Val Loss: 13732.5615
Epoch 7000/10000, Train Loss: 8438.5303, Val Loss: 13892.8955
Epoch 7500/10000, Train Loss: 7888.6548, Val Loss: 14010.3457
Epoch 8000/10000, Train Loss: 7433.0933, Val Loss: 14550.792